# 1단계: HTML 구조 만들기
  1. HTML 파일 (index.html)

In [59]:
import os
import shutil
import subprocess
import sys

# 데스크탑 경로 가져오기
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")

# 프로젝트 폴더 경로
project_folder = os.path.join(desktop_path, "FamilyAlbumProject")

# 기존 폴더 삭제
if os.path.exists(project_folder):
    shutil.rmtree(project_folder)

# 폴더 생성
os.makedirs(project_folder, exist_ok=True)
os.makedirs(os.path.join(project_folder, 'uploads'), exist_ok=True)
os.makedirs(os.path.join(project_folder, 'public'), exist_ok=True)
os.makedirs(os.path.join(project_folder, 'public/images'), exist_ok=True)

# server.js 내용
server_js_content = '''const express = require('express');
const multer = require('multer');
const path = require('path');
const cors = require('cors');
const detectPort = require('detect-port');
const fs = require('fs');

const app = express();
const DEFAULT_PORT = 3000;

app.use(cors());
app.use(express.static(path.join(__dirname, 'public')));
app.use('/uploads', express.static(path.join(__dirname, 'uploads')));
app.use(express.json());
app.use(express.urlencoded({ extended: true }));

const storage = multer.diskStorage({
    destination: function (req, file, cb) {
        const folder = req.body.folder || 'default';
        const uploadPath = path.join(__dirname, 'uploads', folder);
        if (!fs.existsSync(uploadPath)) {
            fs.mkdirSync(uploadPath, { recursive: true });
        }
        cb(null, uploadPath);
    },
    filename: function (req, file, cb) {
        cb(null, Date.now() + path.extname(file.originalname));
    }
});

const upload = multer({ storage: storage });

app.post('/upload', upload.array('files'), (req, res) => {
    const folder = req.body.folder || 'default';
    const theme = req.body.theme;
    const date = req.body.date;

    res.json({
        message: 'Files uploaded successfully',
        folder: folder,
        files: req.files.map(file => ({
            originalname: file.originalname,
            filename: file.filename,
            mimetype: file.mimetype,
            path: file.path,
            theme: theme,
            date: date
        }))
    });
});

app.get('/files', (req, res) => {
    const folder = req.query.folder || 'default';
    const uploadPath = path.join(__dirname, 'uploads', folder);
    fs.readdir(uploadPath, (err, files) => {
        if (err) {
            res.status(500).send('Server Error');
            return;
        }
        res.json(files);
    });
});

app.delete('/files/:folder/:filename', (req, res) => {
    const folder = req.params.folder;
    const filename = req.params.filename;
    fs.unlink(path.join(__dirname, 'uploads', folder, filename), err => {
        if (err) {
            res.status(500).send('Server Error');
            return;
        }
        res.json({ message: 'File deleted successfully' });
    });
});

app.post('/move', (req, res) => {
    const { folder, files, newFolder } = req.body;
    const newFolderPath = path.join(__dirname, 'uploads', newFolder);
    if (!fs.existsSync(newFolderPath)) {
        fs.mkdirSync(newFolderPath, { recursive: true });
    }
    files.forEach(filename => {
        const oldPath = path.join(__dirname, 'uploads', folder, filename);
        const newPath = path.join(newFolderPath, filename);
        fs.renameSync(oldPath, newPath);
    });
    res.json({ message: 'Files moved successfully' });
});

app.get('/', (req, res) => {
    res.sendFile(path.join(__dirname, 'public', 'index.html'));
});

detectPort(DEFAULT_PORT, (err, _port) => {
    if (err) {
        console.log(err);
    }
    const port = _port === DEFAULT_PORT ? DEFAULT_PORT : _port;
    app.listen(port, () => {
        console.log(`Server is running on http://localhost:${port}`);
    });
});
'''

# index.html 내용
index_html_content = '''<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>가족 앨범</title>
    <link rel="stylesheet" href="styles.css">
</head>
<body>
    <div class="container">
        <h1>가족 앨범</h1>
        <form id="passwordForm">
            <label for="passwordInput">비밀번호를 입력하세요:</label>
            <input type="password" id="passwordInput" required>
            <button type="submit">확인</button>
        </form>
        <button id="logoutButton" style="display:none;">로그아웃</button>
        <div id="adminSection" style="display:none;">
            <h2>미디어 업로드</h2>
            <form id="uploadForm">
                <label for="fileInput">사진 또는 동영상을 선택하세요:</label>
                <input type="file" id="fileInput" accept="image/*,video/*" multiple>
                <div id="previewContainer"></div>
                <label for="folderInput">폴더:</label>
                <input type="text" id="folderInput" placeholder="폴더명을 입력하세요">
                <label for="themeInput">테마:</label>
                <select id="themeInput">
                    <option value="일상">일상</option>
                    <option value="가족모임">가족모임</option>
                    <option value="가족여행">가족여행</option>
                    <option value="이벤트">이벤트</option>
                </select>
                <label for="dateInput">날짜:</label>
                <input type="date" id="dateInput">
                <button type="submit">업로드</button>
            </form>
        </div>
        <div id="gallery" style="display:none;">
            <h2>갤러리</h2>
            <div id="filters">
                <label for="filterFolder">폴더 필터:</label>
                <input type="text" id="filterFolder" placeholder="폴더명을 입력하세요">
                <label for="filterTheme">테마별 필터:</label>
                <select id="filterTheme">
                    <option value="">전체</option>
                    <option value="일상">일상</option>
                    <option value="가족모임">가족모임</option>
                    <option value="가족여행">가족여행</option>
                    <option value="이벤트">이벤트</option>
                </select>
                <label for="filterDate">날짜별 필터:</label>
                <input type="date" id="filterDate">
                <button id="applyFilters">필터 적용</button>
            </div>
            <button id="deleteSelected">선택 삭제</button>
            <button id="moveSelected">선택 이동</button>
            <div id="filterResults"></div>
            <div id="mediaDisplay"></div>
        </div>
    </div>
    <div id="lightbox" class="lightbox">
        <span class="close">&times;</span>
        <img class="lightbox-content" id="lightboxImage">
        <video controls class="lightbox-content" id="lightboxVideo"></video>
    </div>
    <script src="script.js"></script>
</body>
</html>'''

# styles.css 내용
styles_css_content = '''body {
    font-family: Arial, sans-serif;
    background-image: url('images/background.jpg');
    background-size: cover;
    background-attachment: fixed;
    color: #333;
    margin: 0;
    padding: 0;
    display: flex;
    justify-content: center;
    align-items: center;
    height: 100vh;
}

.container {
    max-width: 800px;
    margin: 0 auto;
    padding: 20px;
    background-color: rgba(255, 255, 255, 0.9);
    box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
    border-radius: 10px;
    text-align: center;  /* 센터 정렬 추가 */
}

h1, h2 {
    color: #4CAF50;
    text-align: center;
}

form {
    margin: 20px 0;
    text-align: center;
}

label {
    display: block;
    margin-top: 10px;
}

input[type="text"], input[type="date"], input[type="password"], input[type="file"], select, button {
    width: 100%;
    padding: 10px;
    margin-top: 5px;
    border: 1px solid #ccc;
    border-radius: 5px;
    font-size: 1em;
}

button {
    background-color: #4CAF50;
    color: white;
    cursor: pointer;
    font-size: 1em;
    padding: 10px 20px;
}

button:hover {
    background-color: #45a049;
}

#gallery {
    margin-top: 20px;
    height: 400px;
    overflow-y: auto;
}

#mediaDisplay {
    display: flex;
    flex-wrap: wrap;
    justify-content: center;
    gap: 10px;
}

#mediaDisplay img, #mediaDisplay video {
    max-width: 150px;
    height: auto;
    border: 2px solid #ddd;
    padding: 5px;
    display: block;
    margin: 10px;
    border-radius: 10px;
    cursor: pointer;
}

#mediaDisplay .media-container {
    position: relative;
}

#mediaDisplay .media-container input[type="checkbox"] {
    position: absolute;
    top: 10px;
    left: 10px;
    width: 20px;
    height: 20px;
    z-index: 100;
}

#filters {
    margin-bottom: 20px;
    text-align: center;
}

#filters select, #filters input {
    margin-right: 10px;
    padding: 5px;
    border: 1px solid #ccc;
    border-radius: 5px;
}

.download-button, .delete-button {
    display: block;
    margin: 10px auto;
    padding: 10px 20px;
    background-color: #4CAF50;
    color: white;
    text-align: center;
    text-decoration: none;
    border-radius: 5px;
    cursor: pointer;
}

.delete-button {
    background-color: #f44336;
}

.delete-button:hover {
    background-color: #d32f2f;
}

#filterResults {
    text-align: center;
    margin-bottom: 20px;
}

#filterResults ul {
    list-style-type: none;
    padding: 0;
}

#filterResults li {
    display: inline-block;
    margin: 5px;
    padding: 10px;
    background-color: #f1f1f1;
    border-radius: 5px;
    cursor: pointer;
    border: 1px solid #ccc;
}

#filterResults li:hover {
    background-color: #ddd;
}

.lightbox {
    display: none;
    position: fixed;
    z-index: 1000;
    padding-top: 60px;
    left: 0;
    top: 0;
    width: 100%;
    height: 100%;
    overflow: auto;
    background-color: rgba(0,0,0,0.9);
}

.lightbox-content {
    margin: auto;
    display: block;
    max-width: 80%;
    max-height: 80%;
}

.close {
    position: absolute;
    top: 30px;
    right: 35px;
    color: #fff;
    font-size: 40px;
    font-weight: bold;
    transition: 0.3s;
}

.close:hover,
.close:focus {
    color: #bbb;
    text-decoration: none;
    cursor: pointer;
}
'''

# script.js 내용
script_js_content = '''const adminPassword = 'admin123';
const guestPassword = 'guest123';
let approvedMedia = [];

document.getElementById('passwordForm').addEventListener('submit', function(event) {
    event.preventDefault();
    const password = document.getElementById('passwordInput').value;
    
    if (password === adminPassword || password === guestPassword) {
        document.getElementById('passwordForm').style.display = 'none';
        document.querySelector('.container').style.maxWidth = '100%';
        document.getElementById('adminSection').style.display = 'block';
        document.getElementById('gallery').style.display = 'block';
        document.getElementById('logoutButton').style.display = 'block';
        window.scrollTo(0, 0);  // 스크롤을 위로 올리기
        fetchFiles();
    } else {
        alert('비밀번호가 올바르지 않습니다.');
    }
});

document.getElementById('logoutButton').addEventListener('click', function() {
    document.getElementById('passwordForm').style.display = 'block';
    document.querySelector('.container').style.maxWidth = '800px';
    document.getElementById('adminSection').style.display = 'none';
    document.getElementById('gallery').style.display = 'none';
    document.getElementById('logoutButton').style.display = 'none';
    window.scrollTo(0, 0);  // 스크롤을 위로 올리기
});

document.getElementById('fileInput').addEventListener('change', function(event) {
    const files = event.target.files;
    const previewContainer = document.getElementById('previewContainer');
    previewContainer.innerHTML = '';
    
    for (let file of files) {
        const reader = new FileReader();
        reader.onload = function(e) {
            const img = document.createElement('img');
            img.src = e.target.result;
            img.style.maxWidth = '100px';
            img.style.margin = '10px';
            previewContainer.appendChild(img);
        }
        reader.readAsDataURL(file);
    }
});

document.getElementById('uploadForm').addEventListener('submit', function(event) {
    event.preventDefault();
    
    const formData = new FormData();
    const files = document.getElementById('fileInput').files;
    const folder = document.getElementById('folderInput').value;
    const theme = document.getElementById('themeInput').value;
    const date = document.getElementById('dateInput').value;
    
    formData.append('folder', folder);
    formData.append('theme', theme);
    formData.append('date', date);
    
    for (let file of files) {
        formData.append('files', file);
    }
    
    fetch('/upload', {
        method: 'POST',
        body: formData
    })
    .then(response => response.json())
    .then(data => {
        fetchFiles();
    })
    .catch(error => {
        console.error('Error uploading files:', error);
    });
});

document.getElementById('applyFilters').addEventListener('click', function() {
    const filterFolder = document.getElementById('filterFolder').value;
    const filterTheme = document.getElementById('filterTheme').value;
    const filterDate = document.getElementById('filterDate').value;
    const mediaDisplay = document.getElementById('mediaDisplay');
    const filterResults = document.getElementById('filterResults');
    
    filterResults.innerHTML = '';
    mediaDisplay.innerHTML = '';
    
    const filteredMedia = approvedMedia.filter(mediaContainer => {
        const mediaElement = mediaContainer.querySelector('img, video');
        const matchesFolder = filterFolder ? mediaElement.dataset.folder === filterFolder : true;
        const matchesTheme = filterTheme ? mediaElement.dataset.theme === filterTheme : true;
        const matchesDate = filterDate ? mediaElement.dataset.date === filterDate : true;
        return matchesFolder && matchesTheme && matchesDate;
    });
    
    if (filteredMedia.length > 0) {
        const resultList = document.createElement('ul');
        filteredMedia.forEach((mediaContainer, index) => {
            const listItem = document.createElement('li');
            listItem.textContent = `미디어 ${index + 1}`;
            listItem.addEventListener('click', () => {
                mediaDisplay.innerHTML = '';
                mediaDisplay.appendChild(mediaContainer);
            });
            resultList.appendChild(listItem);
        });
        filterResults.appendChild(resultList);
    } else {
        filterResults.textContent = '일치하는 미디어가 없습니다.';
    }
});

document.getElementById('deleteSelected').addEventListener('click', function() {
    const selectedMedia = Array.from(document.querySelectorAll('#mediaDisplay .media-container input[type="checkbox"]:checked'));
    selectedMedia.forEach(checkbox => {
        const mediaContainer = checkbox.closest('.media-container');
        const folder = mediaContainer.dataset.folder;
        const filename = mediaContainer.dataset.filename;
        deleteFile(folder, filename);
    });
});

document.getElementById('moveSelected').addEventListener('click', function() {
    const newFolder = prompt('새 폴더명을 입력하세요:');
    if (newFolder) {
        const selectedMedia = Array.from(document.querySelectorAll('#mediaDisplay .media-container input[type="checkbox"]:checked'));
        const files = selectedMedia.map(checkbox => checkbox.closest('.media-container').dataset.filename);
        const folder = selectedMedia[0].closest('.media-container').dataset.folder;
        moveFiles(folder, files, newFolder);
    }
});

function fetchFiles() {
    const filterFolder = document.getElementById('filterFolder').value;
    const query = filterFolder ? `?folder=${filterFolder}` : '';
    fetch(`/files${query}`)
    .then(response => response.json())
    .then(files => {
        const mediaDisplay = document.getElementById('mediaDisplay');
        mediaDisplay.innerHTML = '';
        approvedMedia = [];

        files.forEach(filename => {
            const mediaContainer = document.createElement('div');
            mediaContainer.classList.add('media-container');
            let mediaElement;

            if (filename.endsWith('.jpg') || filename.endsWith('.jpeg') || filename.endsWith('.png') || filename.endsWith('.gif')) {
                mediaElement = document.createElement('img');
                mediaElement.src = `/uploads/${filterFolder}/${filename}`;
                mediaElement.dataset.folder = filterFolder;
                mediaElement.dataset.filename = filename;
                mediaElement.dataset.theme = 'N/A';
                mediaElement.dataset.date = 'N/A';
                mediaElement.addEventListener('click', function() {
                    openLightbox(mediaElement);
                });
            } else if (filename.endsWith('.mp4') || filename.endsWith('.mov') || filename.endsWith('.avi')) {
                mediaElement = document.createElement('video');
                mediaElement.controls = true;
                mediaElement.src = `/uploads/${filterFolder}/${filename}`;
                mediaElement.dataset.folder = filterFolder;
                mediaElement.dataset.filename = filename;
                mediaElement.dataset.theme = 'N/A';
                mediaElement.dataset.date = 'N/A';
                mediaElement.addEventListener('click', function() {
                    openLightbox(mediaElement);
                });
            }
            
            const checkbox = document.createElement('input');
            checkbox.type = 'checkbox';

            const downloadButton = document.createElement('a');
            downloadButton.href = mediaElement.src;
            downloadButton.download = filename;
            downloadButton.className = 'download-button';
            downloadButton.innerText = '다운로드';

            const deleteButton = document.createElement('button');
            deleteButton.className = 'delete-button';
            deleteButton.innerText = '삭제';
            deleteButton.addEventListener('click', function() {
                deleteFile(filterFolder, filename);
            });

            mediaContainer.appendChild(checkbox);
            mediaContainer.appendChild(mediaElement);
            mediaContainer.appendChild(downloadButton);
            mediaContainer.appendChild(deleteButton);

            approvedMedia.push(mediaContainer);
            mediaDisplay.appendChild(mediaContainer);
        });
    })
    .catch(error => {
        console.error('Error fetching files:', error);
    });
}

function deleteFile(folder, filename) {
    fetch(`/files/${folder}/${filename}`, {
        method: 'DELETE'
    })
    .then(response => response.json())
    .then(data => {
        fetchFiles();
    })
    .catch(error => {
        console.error('Error deleting file:', error);
    });
}

function moveFiles(folder, files, newFolder) {
    fetch('/move', {
        method: 'POST',
        headers: {
            'Content-Type': 'application/json'
        },
        body: JSON.stringify({ folder, files, newFolder })
    })
    .then(response => response.json())
    .then(data => {
        fetchFiles();
    })
    .catch(error => {
        console.error('Error moving files:', error);
    });
}

function openLightbox(mediaElement) {
    const lightbox = document.getElementById('lightbox');
    const lightboxImage = document.getElementById('lightboxImage');
    const lightboxVideo = document.getElementById('lightboxVideo');

    if (mediaElement.tagName === 'IMG') {
        lightboxImage.src = mediaElement.src;
        lightboxImage.style.display = 'block';
        lightboxVideo.style.display = 'none';
    } else if (mediaElement.tagName === 'VIDEO') {
        lightboxVideo.src = mediaElement.src;
        lightboxVideo.style.display = 'block';
        lightboxImage.style.display = 'none';
    }

    lightbox.style.display = 'block';
}

document.querySelector('.close').addEventListener('click', function() {
    document.getElementById('lightbox').style.display = 'none';
    document.getElementById('lightboxImage').src = '';
    document.getElementById('lightboxVideo').src = '';
});
'''

# 파일 생성 및 내용 쓰기
with open(os.path.join(project_folder, 'server.js'), 'w', encoding='utf-8') as file:
    file.write(server_js_content)

with open(os.path.join(project_folder, 'public/index.html'), 'w', encoding='utf-8') as file:
    file.write(index_html_content)

with open(os.path.join(project_folder, 'public/styles.css'), 'w', encoding='utf-8') as file:
    file.write(styles_css_content)

with open(os.path.join(project_folder, 'public/script.js'), 'w', encoding='utf-8') as file:
    file.write(script_js_content)

# 랜딩페이지 배경이미지 추가
background_image_url = 'https://www.w3schools.com/w3images/lights.jpg'  # 적절한 배경 이미지 URL
background_image_path = os.path.join(project_folder, 'public/images/background.jpg')
subprocess.run(["curl", background_image_url, "-o", background_image_path])

print(f"프로젝트 폴더와 파일들이 {project_folder} 경로에 생성되었습니다.")

# Node.js 프로젝트 초기화
subprocess.run(["npm", "init", "-y"], cwd=project_folder)

# 필요한 패키지 설치
subprocess.run(["npm", "install", "express", "multer", "cors", "detect-port"], cwd=project_folder)

# Node.js 서버 실행
subprocess.Popen(["node", os.path.join(project_folder, "server.js")])


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 20461  100 20461    0     0  95995      0 --:--:-- --:--:-- --:--:-- 96061


프로젝트 폴더와 파일들이 /Users/heebonpark/Desktop/FamilyAlbumProject 경로에 생성되었습니다.
Wrote to /Users/heebonpark/Desktop/FamilyAlbumProject/package.json:

{
  "name": "familyalbumproject",
  "version": "1.0.0",
  "main": "server.js",
  "scripts": {
    "test": "echo \"Error: no test specified\" && exit 1",
    "start": "node server.js"
  },
  "keywords": [],
  "author": "",
  "license": "ISC",
  "description": ""
}




added 94 packages, and audited 95 packages in 6s

13 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities


<Popen: returncode: None args: ['node', '/Users/heebonpark/Desktop/FamilyAlb...>

Server is running on http://localhost:60268
